# Welcome to the Flor Tutorial!


* Using Flor for lightweight versioning and tracking of ML/AI experiments.

* How to collaborate using Flor 

* What happens with more detailed Flor pipelines

# Part 1: Annotating and Versioning

## Prepare your environment before starting the activities.

We're going to start by importing Flor and letting it know the name of our notebook.

In [ ]:
# Import Flor
import flor

# If the notebook name has not already been set, you are able to set the name in code. 
flor.setNotebookName('pinterest_demo_nc.ipynb')

## Sentiment Analysis

* We're going to train a model to predict the sentiment -- positive, or negative -- of English phrases.

The pipeline is pretty standard: 

* load the dataset
* do some light preprocessing 
* do a train/test split
* train the model
* validate the model

#### Experiment 1: Random Forest Classifier with 5 estimators.

In [ ]:
# Import standard libraries
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load the Data
movie_reviews = pd.read_json('data.json')

# Do light preprocessing
movie_reviews['rating'] = movie_reviews['rating'].map(lambda x: 0 if x < 5 else 1)

# Do train/test split
X_tr, X_te, y_tr, y_te = train_test_split(movie_reviews['text'], movie_reviews['rating'], 
                                          test_size=0.20, random_state=92)

# Vectorize the English sentences
vectorizer = TfidfVectorizer()
vectorizer.fit(X_tr)
X_tr = vectorizer.transform(X_tr)
X_te = vectorizer.transform(X_te)

# Fit the model              ##############
clf = RandomForestClassifier(n_estimators=5).fit(X_tr, y_tr)
                             ##############
y_pred = clf.predict(X_te)

# Validate the predictions
c = classification_report(y_te, y_pred)

print(c)

## What did we learn? 

### Versioning 

* We did not overwrite the previous Jupyter cell to preserve the history of our experimentation 

### Experiment Annotation

* We used inline documentation and code comments to highlight key features 

### but... this relied on the user adhering to "best practices"

## Sentiment Analysis with Flor


In [ ]:
###################################
@flor.track_action('demo')
def split_train_and_eval(**kwargs):
###################################
    import pandas as pd

    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import classification_report
    movie_reviews = pd.read_json('data.json')
    movie_reviews['rating'] = movie_reviews['rating'].map(lambda x: 0 if x < 5 else 1)

    X_tr, X_te, y_tr, y_te = train_test_split(movie_reviews['text'], movie_reviews['rating'], 
                                              test_size=0.20, random_state=92)

    vectorizer = TfidfVectorizer()
    vectorizer.fit(X_tr)
    X_tr = vectorizer.transform(X_tr)
    X_te = vectorizer.transform(X_te)
    clf = RandomForestClassifier(n_estimators=5).fit(X_tr, y_tr)
    
    y_pred = clf.predict(X_te)

    score = clf.score(X_te, y_te)
    c = classification_report(y_te, y_pred)

    print(c)
    
    #######################
    return {'score': score}
    #######################

# This is how we run the pipeline
split_train_and_eval()

In [ ]:
flor.Experiment('demo').summarize()

### Now let's try evaluating n_estimators at 7 again

## Hyper-parameter sweeps in Flor

In [ ]:
@flor.track_action('demo')
                         ############  #########
def split_train_and_eval(n_estimators, max_depth, **kwargs):
                         ############  #########
    import pandas as pd

    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import classification_report
    movie_reviews = pd.read_json('data.json')
    movie_reviews['rating'] = movie_reviews['rating'].map(lambda x: 0 if x < 5 else 1)

    X_tr, X_te, y_tr, y_te = train_test_split(movie_reviews['text'], movie_reviews['rating'], 
                                              test_size=0.20, random_state=92)

    vectorizer = TfidfVectorizer()
    vectorizer.fit(X_tr)
    X_tr = vectorizer.transform(X_tr)
    X_te = vectorizer.transform(X_te)         ############            #########
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth).fit(X_tr, y_tr)
                                              ############            #########
    y_pred = clf.predict(X_te)

    score = clf.score(X_te, y_te)
    c = classification_report(y_te, y_pred)

    print(c)
    
    return {'score': score}
    
# This is how we run the pipeline
split_train_and_eval(n_estimators=7, max_depth=[10, 100])

In [ ]:
flor.Experiment('demo').summarize()

# Part 2: Sharing and Composition

## Interpreting someone else's work in Flor

* Our friend Bob is a Flor user and already did some preprocessing work in his experiment 'bob_preproc'

In [ ]:
flor.Experiment('bob_preproc').summarize()

We can see there are two past versions of the experiment bob_preproc: first_preproc, and second_preproc

In [ ]:
flor.Experiment('bob_preproc').plot('first_preproc')

We can now see the structure of the Flor Plan for Bob's preprocessing expriment. We see that there are four (4) rectangles: 
* Inputs: `preprocess`, `data_loc`
* Outputs: `intermediate_X`, and `intermediate_y`.

In [ ]:
flor.Experiment('bob_preproc').plot('second_preproc')

We see that both node-link diagrams look the same. This means that the structure of the different experiment versions is the same; however, it is very likely that the contents of the computation graph differ.

In [ ]:
flor.Experiment('bob_preproc').diff('first_preproc', 'second_preproc')

If we want to inspect the differences in more detail, we can do so by controlling the flags to `diff`. Below we add the flag `--minimal`, but you're welcome to try [other flags](https://git-scm.com/docs/git-diff).

In [ ]:
flor.Experiment('bob_preproc').diff('first_preproc', 'second_preproc', flags=['--minimal'])

## Beyond the Basic Flor API: Flor's Extended API
We previously used the `flor.track_action` decorator to wrap a pipeline and execute it with a minimum of extra Flor wrapper code. `track_action` infers a simple underlying Flor Plan for our code. We refer to functions wrapped with `track_action` as Flor's *basic* API. 

We wrap our functions with the `flor.func` decorator. This requires the developer to explicitly declare a Flor Plan. This allows us to represent pipelines with __many Actions__, and enables us to __re-use the Artifacts of someone else's experiment__.

## Using Bob's work in Flor

In [ ]:
flor.Experiment('bob_preproc').summarize()

### Here we define the split_train_and_eval function. Running this cell will not execute the experiment because we will need to specify a Flor plan. 

In [ ]:
##########
@flor.func
##########               ##############  ##############
def split_train_and_eval(intermediate_X, intermediate_y, n_estimators, max_depth, **kwargs):
                         ##############  ##############
    import pandas as pd
    import json

    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import classification_report
            
              ##############
    with open(intermediate_X) as json_data:
              ##############
        X = json.load(json_data)
        json_data.close()
        
              ##############
    with open(intermediate_y) as json_data:
              ##############
        y = json.load(json_data)
        json_data.close()

    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.20, random_state=92)

    vectorizer = TfidfVectorizer()
    vectorizer.fit(X_tr)
    X_tr = vectorizer.transform(X_tr)
    X_te = vectorizer.transform(X_te)
    
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth).fit(X_tr, y_tr)
    
    y_pred = clf.predict(X_te)

    score = clf.score(X_te, y_te)
    print(score)
    
    return {'score': score}

### Now we define the Flor plan 

* Notice that we are able to maintain context of multiple experiments 

In [ ]:
with flor.Experiment('bob_preproc') as bob, flor.Experiment('demo') as ex:
    # This is how we tell Flor we will be using Bob's derived artifacts
    data_x = bob.artifact('data_clean_X.json', 'intermediate_X', label="first_preproc")
    data_y = bob.artifact('data_clean_y.json', 'intermediate_y', label="first_preproc")
    
    # This is how we specify the "literals" (parameters) of the Flor Plan for our experiments     
    n_estimators = ex.literal(7, 'n_estimators') # The best parameter from the previous experiment
    max_depth = ex.literal(100, 'max_depth')
                                                                
    # This is where we put the code we declared in the cell above
    do_split_train_and_eval = ex.action(split_train_and_eval, [data_x, data_y, n_estimators, max_depth])
    score = ex.literal(name='score', parent=do_split_train_and_eval)


### Let's verify that everything looks as it should. 

In [ ]:
score.plot()

In [ ]:
score.pull('fifth_pull')

In [ ]:
flor.Experiment('demo').summarize()

### Despite using the best configuration from the previous experiment, our score dropped to 65% :( 

* The only change we made was using Bob's preprocessed data and we tried out the first version already. Let's try using the second version. 

In [ ]:
flor.Experiment('bob_preproc').summarize()

In [ ]:
data_x.version = "second_preproc"
data_y.version = "second_preproc"

In [ ]:
score.plot()

In [ ]:
score.pull('sixth_pull')

In [ ]:
flor.Experiment('demo').summarize()

# Detailed Flor Pipelines

In [ ]:
@flor.func
def split(intermediate_X, intermediate_y, test_size, random_state, 
          X_train, X_test, y_train, y_test, **kwargs):
    import json
    from sklearn.model_selection import train_test_split
    
    # Read the inputs
    with open(intermediate_X) as json_data:
        X = json.load(json_data)
        json_data.close()
        
    with open(intermediate_y) as json_data:
        y = json.load(json_data)
        json_data.close()
        
    # Split the data
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # Write the outputs
    with open(X_train, 'w') as f:
        json.dump(X_tr, f)
    with open(X_test, 'w') as f:
        json.dump(X_te, f)
    with open(y_train, 'w') as f:
        json.dump(y_tr, f)
    with open(y_test, 'w') as f:
        json.dump(y_te, f)

In [ ]:
@flor.func
def train(X_train, y_train, n_estimators, max_depth, model, vectorizer, **kwargs):
    import pandas as pd
    import json
    import cloudpickle

    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.ensemble import RandomForestClassifier
    
    # Read the inputs
    with open(X_train, 'r') as f:
        X_tr = json.load(f)
    with open(y_train, 'r') as f:
        y_tr = json.load(f)
    
    # Fit the vectorizer
    vec = TfidfVectorizer()
    vec.fit(X_tr)
    
    # Transform the training data
    X_tr = vec.transform(X_tr)
    
    # Train the model
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth).fit(X_tr, y_tr)
    
    # Write the output
    with open(model, 'wb') as f:
        cloudpickle.dump(clf, f)
    with open(vectorizer, 'wb') as f:
        cloudpickle.dump(vec, f)

In [ ]:
@flor.func
def eval(X_test, y_test, model, vectorizer, **kwargs):
    import pandas as pd
    import json
    import cloudpickle
    
    # Read the inputs
    with open(X_test, 'r') as f:
        X_te = json.load(f)
    with open(y_test, 'r') as f:
        y_te = json.load(f)
    with open(model, 'rb') as f:
        clf = cloudpickle.load(f)
    with open(vectorizer, 'rb') as f:
        vec = cloudpickle.load(f)
    
    # Test the model
    X_te = vec.transform(X_te)
    score = clf.score(X_te, y_te)
    
    print(score)
    
    # Return the score
    return {'score': score}

In [ ]:
with flor.Experiment('bob_preproc') as bob, flor.Experiment('demo') as ex:
    # This is how we tell Flor we will be using Bob's derived artifacts
    data_x = bob.artifact('data_clean_X.json', 'intermediate_X', label="second_preproc")
    data_y = bob.artifact('data_clean_y.json', 'intermediate_y', label="second_preproc")
    
    # Here we declare all the static literals
    random_state = ex.literal(92, 'random_state')
    test_size = ex.literal(0.20, 'test_size')
    n_estimators = ex.literal(7, 'n_estimators') 
    max_depth = ex.literal(100, 'max_depth')
    
    # Now we connect the Flor Plan
    do_split = ex.action(func=split, in_artifacts=[data_x, data_y, test_size, random_state])
    X_train = ex.artifact(loc='X_train.json', name='X_train', parent=do_split)
    X_test = ex.artifact(loc='X_test.json', name='X_test', parent=do_split)
    y_train = ex.artifact(loc='y_train.json', name='y_train', parent=do_split)
    y_test = ex.artifact(loc='y_test.json', name='y_test', parent=do_split)
    
    do_train = ex.action(func=train, in_artifacts=[X_train, y_train, n_estimators, max_depth])
    model = ex.artifact(loc='model.pkl', name='model', parent=do_train)
    vectorizer = ex.artifact(loc='vectorizer.pkl', name='vectorizer', parent=do_train)
    
    do_eval = ex.action(func=eval, in_artifacts=[X_test, y_test, model, vectorizer])
    score = ex.literal(name='score', parent=do_eval)

In [ ]:
score.plot()

In [ ]:
score.pull('seventh_pull')

In [ ]:
flor.Experiment('demo').summarize()

## Reusing models from previous Flor experiments

In [ ]:
with flor.Experiment('demo') as ex:
    model = ex.artifact('model.pkl', 'model', label='seventh_pull')
    vectorizer = ex.artifact('vectorizer.pkl', 'vectorizer', label='seventh_pull')
model = model.peek()
vec = vectorizer.peek()

In [ ]:
PROMPT = "What's on your mind? "

phrase = input(PROMPT)
while phrase[0:len('nothing')].lower() != 'nothing':
    phrase = vec.transform([phrase,])
    positive = model.predict(phrase)
    if positive:
        print('Happy to hear that!\n'.format(phrase))
    else:
        print("Sorry about that...\n")
    phrase = input(PROMPT)
print('you said nothing.')